In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import joblib
import json
import warnings
warnings.filterwarnings('ignore')

# ==============================================================================
# 1. CARREGAMENTO E PRÉ-PROCESSAMENTO DOS DADOS
# ==============================================================================

print("=" * 80)
print("SISTEMA DE DIAGNÓSTICO MÉDICO - STACKING ENSEMBLE")
print("=" * 80)

# Carregar dataset
print("\n[1/6] Carregando dataset...")
df = pd.read_csv('Final_Augmented_dataset_Diseases_and_Symptoms.csv')
# Remover doenças raras (ocorrência única)
df = df[df['diseases'].isin(df['diseases'].value_counts()[df['diseases'].value_counts() > 1].index)]


print(f"   ✓ Dataset carregado: {df.shape[0]} amostras, {df.shape[1]} colunas")
print(f"   ✓ Número de doenças únicas: {df['diseases'].nunique()}")
print(f"   ✓ Número de sintomas: {df.shape[1] - 1}")

# Separar features e target
X = df.drop('diseases', axis=1)
y = df['diseases']

# Codificar labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

print(f"\n   Classes de doenças codificadas: 0 a {len(label_encoder.classes_) - 1}")

# ==============================================================================
# 2. DIVISÃO DOS DADOS
# ==============================================================================

print("\n[2/6] Dividindo dados em treino e teste...")
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, 
    test_size=0.2, 
    random_state=42, 
    stratify=y_encoded
)

print(f"   ✓ Treino: {X_train.shape[0]} amostras")
print(f"   ✓ Teste: {X_test.shape[0]} amostras")

# ==============================================================================
# 3. DEFINIÇÃO DO MODELO STACKING
# ==============================================================================

print("\n[3/6] Configurando Stacking Ensemble...")

# Base learners (modelos de primeira camada)
base_learners = [
    ('rf', RandomForestClassifier(
        n_estimators=100, 
        max_depth=20,
        min_samples_split=5,
        min_samples_leaf=2,
        random_state=42,
        n_jobs=-1,
        max_features='sqrt'
    )),
    ('gb', GradientBoostingClassifier(
        n_estimators=100,
        max_depth=10,
        learning_rate=0.1,
        subsample=0.8,
        random_state=42
    )),
    ('nb', GaussianNB())
]

# Meta learner (modelo de segunda camada)
meta_learner = LogisticRegression(
    max_iter=1000,
    random_state=42,
    n_jobs=-1,
    solver='lbfgs'
)

# Criar Stacking Classifier
stacking_model = StackingClassifier(
    estimators=base_learners,
    final_estimator=meta_learner,
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
    n_jobs=-1,
    verbose=1
)

print("   ✓ Base Learners:")
print("      - Random Forest (100 árvores)")
print("      - Gradient Boosting (100 árvores)")
print("      - Naive Bayes")
print("   ✓ Meta Learner: Logistic Regression")
print("   ✓ Cross-Validation: 5-fold Stratified")

# ==============================================================================
# 4. TREINAMENTO DO MODELO
# ==============================================================================

print("\n[4/6] Treinando modelo Stacking...")
print("   (Isso pode levar alguns minutos...)\n")

stacking_model.fit(X_train, y_train)

print("\n   ✓ Treinamento concluído!")

# ==============================================================================
# 5. AVALIAÇÃO DO MODELO
# ==============================================================================

print("\n[5/6] Avaliando modelo...")

# Predições
y_pred_train = stacking_model.predict(X_train)
y_pred_test = stacking_model.predict(X_test)

# Acurácia
train_accuracy = accuracy_score(y_train, y_pred_train)
test_accuracy = accuracy_score(y_test, y_pred_test)

print(f"\n   ✓ Acurácia no treino: {train_accuracy:.4f} ({train_accuracy*100:.2f}%)")
print(f"   ✓ Acurácia no teste:  {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")

# Relatório de classificação (resumido)
print("\n   Relatório de Classificação (Teste):")
print("   " + "-" * 76)
report = classification_report(y_test, y_pred_test, 
                              target_names=label_encoder.classes_,
                              output_dict=True)

# Mostrar métricas médias
print(f"   Precisão média:  {report['weighted avg']['precision']:.4f}")
print(f"   Recall médio:    {report['weighted avg']['recall']:.4f}")
print(f"   F1-Score médio:  {report['weighted avg']['f1-score']:.4f}")

# ==============================================================================
# 6. SALVAR MODELO E ARTEFATOS
# ==============================================================================

print("\n[6/6] Salvando modelo e artefatos...")

# Salvar modelo
joblib.dump(stacking_model, 'stacking_disease_model.pkl')
print("   ✓ Modelo salvo: stacking_disease_model.pkl")

# Salvar label encoder
joblib.dump(label_encoder, 'label_encoder.pkl')
print("   ✓ Label encoder salvo: label_encoder.pkl")

# Salvar lista de features
feature_names = X.columns.tolist()
with open('feature_names.json', 'w') as f:
    json.dump(feature_names, f)
print("   ✓ Features salvas: feature_names.json")

# ==============================================================================
# 7. FUNÇÃO DE PREDIÇÃO COM JSON
# ==============================================================================

def predict_disease_from_json(json_input):
    """
    Prediz doença a partir de um JSON de sintomas.
    
    Args:
        json_input (dict ou str): JSON com sintomas (1 = presente, 0 = ausente)
                                  Sintomas não fornecidos são considerados 0
    
    Returns:
        dict: Dicionário com predição e probabilidades
    """
    # Carregar artefatos
    model = joblib.load('stacking_disease_model.pkl')
    encoder = joblib.load('label_encoder.pkl')
    with open('feature_names.json', 'r') as f:
        features = json.load(f)
    
    # Parse JSON se for string
    if isinstance(json_input, str):
        json_input = json.loads(json_input)
    
    # Criar vetor de features (default = 0)
    feature_vector = np.zeros(len(features))
    
    # Preencher sintomas fornecidos
    for symptom, value in json_input.items():
        if symptom in features:
            idx = features.index(symptom)
            feature_vector[idx] = int(value)
    
    # Reshape para predição
    feature_vector = feature_vector.reshape(1, -1)
    
    # Predição
    prediction = model.predict(feature_vector)[0]
    probabilities = model.predict_proba(feature_vector)[0]
    
    # Decodificar doença
    disease = encoder.inverse_transform([prediction])[0]
    
    # Top 5 doenças mais prováveis
    top_5_indices = np.argsort(probabilities)[-5:][::-1]
    top_5_diseases = [
        {
            'disease': encoder.inverse_transform([idx])[0],
            'probability': float(probabilities[idx])
        }
        for idx in top_5_indices
    ]
    
    return {
        'predicted_disease': disease,
        'confidence': float(probabilities[prediction]),
        'top_5_predictions': top_5_diseases
    }

print("\n   ✓ Função de predição criada: predict_disease_from_json()")

# ==============================================================================
# 8. EXEMPLO DE USO
# ==============================================================================

print("\n" + "=" * 80)
print("EXEMPLO DE PREDIÇÃO")
print("=" * 80)

# Exemplo de JSON com sintomas
example_json = {
    "fever": 1,
    "cough": 1,
    "fatigue": 1,
    "headache": 1,
    "shortness of breath": 1
}

print("\nSintomas de entrada (JSON):")
print(json.dumps(example_json, indent=2))

# Realizar predição
result = predict_disease_from_json(example_json)

print("\n" + "-" * 80)
print("RESULTADO DA PREDIÇÃO:")
print("-" * 80)
print(f"\nDoença Prevista: {result['predicted_disease']}")
print(f"Confiança: {result['confidence']:.4f} ({result['confidence']*100:.2f}%)")

print("\nTop 5 Diagnósticos Mais Prováveis:")
for i, pred in enumerate(result['top_5_predictions'], 1):
    print(f"   {i}. {pred['disease']}: {pred['probability']:.4f} ({pred['probability']*100:.2f}%)")

print("\n" + "=" * 80)
print("SISTEMA PRONTO PARA USO!")
print("=" * 80)
print("\nPara fazer novas predições, use:")
print("   result = predict_disease_from_json(seu_json)")
print("\nExemplo de JSON vazio (todos sintomas = 0):")
print("   result = predict_disease_from_json({})")
print("=" * 80)

# ==============================================================================
# INFORMAÇÕES SOBRE MEMÓRIA
# ==============================================================================

print("\n📊 Uso de Memória:")
print(f"   - Tamanho do modelo em disco: ~{joblib.dump(stacking_model, 'temp.pkl')} bytes")
import os
if os.path.exists('temp.pkl'):
    size_mb = os.path.getsize('temp.pkl') / (1024 * 1024)
    print(f"   - Tamanho aproximado: {size_mb:.2f} MB")
    os.remove('temp.pkl')

print("\n✅ Sistema otimizado para uso consistente e eficiente de memória!")

SISTEMA DE DIAGNÓSTICO MÉDICO - STACKING ENSEMBLE

[1/6] Carregando dataset...
   ✓ Dataset carregado: 246926 amostras, 378 colunas
   ✓ Número de doenças únicas: 754
   ✓ Número de sintomas: 377

   Classes de doenças codificadas: 0 a 753

[2/6] Dividindo dados em treino e teste...
   ✓ Treino: 197540 amostras
   ✓ Teste: 49386 amostras

[3/6] Configurando Stacking Ensemble...
   ✓ Base Learners:
      - Random Forest (100 árvores)
      - Gradient Boosting (100 árvores)
      - Naive Bayes
   ✓ Meta Learner: Logistic Regression
   ✓ Cross-Validation: 5-fold Stratified

[4/6] Treinando modelo Stacking...
   (Isso pode levar alguns minutos...)



Total classes válidas: 754


InvalidParameterError: The 'loss' parameter of SGDClassifier must be a str among {'log_loss', 'squared_hinge', 'huber', 'modified_huber', 'perceptron', 'squared_error', 'epsilon_insensitive', 'hinge', 'squared_epsilon_insensitive'}. Got 'log' instead.